# Aplicando os modelos no conjunto de teste

## Pré-processa dados de teste

In [ ]:
import pickle

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    root_mean_squared_error,
)
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Features selecionadas

caminho_features_selecionadas_all_features = (
    "../data/08_reporting/selected_features_all_features.npy"
)

caminho_features_selecionadas_non_linear_features = (
    "../data/08_reporting/selected_non_linear_features.npy"
)

caminho_features_selecionadas_linear_features = (
    "../data/08_reporting/selected_linear_features.npy"
)

# Caso os dvalores shapa já estejam calculados

caminho_shap_all_features = (
    "../data/08_reporting/shap_values_calculados_all_features.npy"
)

caminho_shap_non_linear_features = "../data/08_reporting/shap_values_calculados.npy"

caminho_shap_linear_features = (
    "../data/08_reporting/shap_values_calculados_linear_features.npy"
)

# Conjunto de treino normalizado para plotar o gráfico

caminho_X_train_all_features = "../data/03_primary/X_train_all_features.npy"

caminho_X_train_non_linear_features = (
    "../data/03_primary/X_train_non_linear_features.npy"
)

caminho_X_train_linear_features = "../data/03_primary/X_train_linear_features.npy"

# Target de treino normalizado

caminho_y_train_all_features = "../data/03_primary/y_train_all_features.npy"

caminho_y_train_non_linear_features = (
    "../data/03_primary/y_train_non_linear_features.npy"
)

caminho_y_train_linear_features = "../data/03_primary/y_train_linear_features.npy"

# Conjunto de teste normalizado para plotar o gráfico

caminho_X_test_shap_all_features = (
    "../data/03_primary/X_test_final_para_shap_all_features.npy"
)

caminho_X_test_shap_non_linear_features = (
    "../data/03_primary/X_test_final_para_shap.npy"
)

caminho_X_test_shap_linear_features = (
    "../data/03_primary/X_test_final_para_shap_linear_features.npy"
)

# Target de teste normalizado

caminho_y_teste_normalizado = "../data/03_primary/y_teste_normalizado.npy"


# shap explainers

caminho_explainer_all_features = (
    "../data/08_reporting/explainer_expected_value_all_features.pkl"
)

caminho_explainer_non_linear_features = (
    "../data/08_reporting/explainer_expected_value.pkl"
)

caminho_explainer_linear_features = (
    "../data/08_reporting/explainer_expected_value_linear_features.pkl"
)

# modelos

caminho_modelo_all_features = "../data/06_models/modelo_notebook_2_all_data.joblib"

caminho_modelo_non_linear_features = "../data/06_models/modelo_notebook_3.joblib"

caminho_modelo_linear_features = "../data/06_models/modelo_notebook_4.joblib"

In [ ]:
teste = pd.read_csv("../data/02_intermediate/teste_b_df.csv")

In [ ]:
teste

In [ ]:
X_test = teste.drop(columns=["r"])
y_test = teste["r"]

## Criando features extras

Todas as features criadas para treinar o modelo devem também ser criadas ao se aplicar os dados de teste. 

In [ ]:
X_test = utils.cria_feature_media_ponderada(X_test)

In [ ]:
X_test

## Criando o MinMaxScaler

- O `MinMaxScaler` é ajustado (`fit`) no conjunto de treino para aprender os valores de Mínimo e Máximo. O **mesmo scaler ajustado** é então usado para **transformar** o conjunto de teste e garantir a coerência na escala dos dados.

- Ajustar o scaler nos dados de teste criaria um **Vazamento de Dados (*Data Leakage*)**, pois o modelo teria acesso, indiretamente, à distribuição dos dados que deveriam ser inéditos, levando a uma avaliação de desempenho irrealista e otimista demais.

In [ ]:
treino = pd.read_csv("../data/02_intermediate/training_b_df.csv")

X_train = treino.drop(columns=["r"])
y_train = treino["r"]

In [ ]:
# 1. Escalonamento das Features (X)
X_scaler = MinMaxScaler()
X_train_scaled = X_scaler.fit_transform(X_train)

# 2. Escalonamento do Target (Y), se for Regressão Contínua
# Reformatar y_train para que o scaler funcione (de Series para 2D array/DataFrame)
y_train_2d = y_train.values.reshape(-1, 1)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train_2d)

## Aplicando o MinMaxScaler() no target

- Para as visualizações das métricas dos modelos, precisa-se aplicar o scaler no target apenas uma vez.

- Abaixo aplica-se o y_scaler nos dados de teste e os salva para sua utilização no notebook de 6_Explicabilidade

In [ ]:
# Escalonamento do Target (Y)
# Reformatar y_train para que o scaler funcione (de Series para 2D array/DataFrame)
y_test_2d = y_test.values.reshape(-1, 1)
y_test_scaled = y_scaler.fit_transform(y_test_2d)

np.save(caminho_y_teste_normalizado, y_train_scaled)

# 1 - Modelo com todas as features

In [ ]:
# Checa se o arquivo foi salvo corretamente
with open(caminho_features_selecionadas_all_features, "rb") as f:
    loaded_data = pickle.load(f)


selected_feature_names = loaded_data["list"]

In [ ]:
selected_feature_names

In [ ]:
X_test_all_features = X_test[selected_feature_names]

In [ ]:
X_test_scaled = X_scaler.fit_transform(X_test_all_features)

In [ ]:
X_test_scaled

In [ ]:
X_test_scaled.max()

In [ ]:
X_test_scaled.min()

## 1.2 - Salvando valores normalizados

As features normalizadas serão necessárias no notebook de explicabilidade dos modelos, por isso salva-se abaixo:

In [ ]:
np.save(caminho_X_test_shap_all_features, X_test_scaled)

# 1.3 - Inferência 

In [ ]:
# Carregamento do modelo (usando o caminho verificado)
best_nn_model = joblib.load(caminho_modelo_all_features)

In [ ]:
# A previsão é feita na escala Z-Score (escalada)
y_pred_test_scaled = best_nn_model.predict(X_test_scaled).reshape(-1, 1)

# Faz a transformação inversa para obter os valores em 'r':
y_pred_test_original = y_scaler.inverse_transform(y_pred_test_scaled)


print("Previsão no Conjunto de Teste concluída com sucesso.")

In [ ]:
X_test_scaled.max()

In [ ]:
y_pred_test_scaled.max()

In [ ]:
X_test_scaled.min()

In [ ]:
y_pred_test_scaled.min()

In [ ]:
y_pred_test_original.max()

In [ ]:
y_pred_test_original.min()

## 1. 4 - Métricas e visualizações

In [ ]:
# Cálculo das 3 Métricas de Avaliação Finais:
mse_test = mean_squared_error(y_test_2d, y_pred_test_original)
rmse_test = root_mean_squared_error(y_test_2d, y_pred_test_original)
mae_test = mean_absolute_error(y_test_2d, y_pred_test_original)
r2_test = r2_score(y_test_2d, y_pred_test_original)

print("\n" + "=" * 50)
print("AVALIAÇÃO DE DESEMPENHO NO CONJUNTO DE TESTE")
print("=" * 50)
print(f"MSE (Erro Quadrático Médio): {mse_test:.8f}")
print(f"RMSE (Erro Absoluto Médio):   {rmse_test:.8f} (Erro médio na unidade de 'r')")
print(f"MAE (Erro Absoluto Médio):   {mae_test:.8f} (Erro médio na unidade de 'r')")
print(f"R2 (Ajuste):                 {r2_test:.4f}")
print("=" * 50)

In [ ]:
# Crie a linha de identidade X=Y
min_val = min(y_test_2d.min(), y_pred_test_original.min())
max_val = max(y_test_2d.max(), y_pred_test_original.max())
ideal_line = np.linspace(min_val, max_val, 100)

plt.figure(figsize=(8, 8))

# 1. Scatter Plot dos Resultados
plt.scatter(y_test_2d, y_pred_test_original, alpha=0.6, s=20, label="Previsões")

# 2. Linha de Identidade (Ajuste Perfeito)
plt.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

plt.title(f"Previsões vs. Valores Reais de r (R² Final: {r2_test:.4f})", fontsize=14)
plt.xlabel("Valores Reais de r", fontsize=12)
plt.ylabel("Valores Previstos de r", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.gca().set_aspect(
    "equal", adjustable="box"
)  # Garante que os eixos tenham a mesma escala
plt.show()

In [ ]:
# Calcule os resíduos (True - Predicted)
residuals = y_test_2d - y_pred_test_original

# Achata o array para 1D (o formato (N,))
residuals_1d = residuals.ravel()

plt.figure(figsize=(8, 6))

# Histograma dos resíduos
plt.hist(residuals_1d, bins=30, edgecolor="black", alpha=0.7)

# Linha vertical em zero (onde o centro do histograma deveria estar)
plt.axvline(x=0, color="red", linestyle="--", linewidth=2, label="Erro Zero")

plt.title("Distribuição dos Resíduos (Erros)", fontsize=14)
plt.xlabel("Resíduo (Real r - Previsto r)", fontsize=12)
plt.ylabel("Frequência", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.show()

## 1.4.1 - Visualização com barra de erros

Os dados utilizados associam 10 valores diferentes das features a um mesmo valor do target. O objetivo é simular os ruídos presentes em observações astronômicas.

A visualização com barras de erros também é feita abaixo utilizando-se os dados de teste.

In [ ]:
# Criando um DataFrame para facilitar a agregação
# Garantindo que y_train e y_pred_original são 1D para o DataFrame
y_test_flat = y_test_2d.ravel()
y_pred_test_flat = y_pred_test_original.ravel()

df_results = pd.DataFrame({"y_test": y_test_flat, "y_test_pred": y_pred_test_flat})

# AGREGAR: Agrupar por 'y_true' e calcular Média e Desvio Padrão para 'y_pred'
# 'y_true' (a variável agrupada) será o índice do novo DataFrame
df_grouped = (
    df_results.groupby("y_test")["y_test_pred"].agg(["mean", "std"]).reset_index()
)

# Renomear as colunas para clareza
df_grouped.columns = ["y_test", "y_test_pred_mean", "y_test_pred_std"]

# Cálculo do Resíduo Padronizado (em módulos, conforme solicitado)
# Evite a divisão por zero, substituindo desvios padrão zero por um valor pequeno (ex: 1e-6)
df_grouped["y_test_pred_std_safe"] = df_grouped["y_test_pred_std"].replace(0, 1e-6)
df_grouped["Standardized_Residual"] = (
    np.abs(df_grouped["y_test"] - df_grouped["y_test_pred_mean"])
    / df_grouped["y_test_pred_std_safe"]
)

# Exibir o resultado da agregação (opcional)
print("Dados Agrupados (Exemplo):")
print(df_grouped.head())

In [ ]:
# --- VISUALIZAÇÃO COM GRÁFICO DE DUPLO EIXO ---
# Cria um espaço de figuras com 2 linhas e 1 coluna, compartilhando o eixo X
fig, (ax1, ax2) = plt.subplots(
    2,
    1,
    figsize=(8, 10),
    sharex=True,  # Os dois gráficos compartilham o mesmo eixo X
    gridspec_kw={"hspace": 0.05},  # Reduz o espaço entre os subplots
    constrained_layout=True,
)

# ==========================================================
# GRÁFICO SUPERIOR: Previsão Média vs. Real (Com Barras de Erro)
# ==========================================================

# 1. Plotar os pontos agregados com Barras de Erro
ax1.errorbar(
    x=df_grouped["y_test"],
    y=df_grouped["y_test_pred_mean"],
    yerr=df_grouped["y_test_pred_std"],
    fmt="o",
    capsize=4,
    alpha=0.7,
    label="Previsão Média ± 1 DP",
)

# 2. Linha de Identidade (Ajuste Perfeito Y=X)
ax1.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

ax1.set_title(f"Análise de Previsão Agregada (R² Final: {r2_test:.4f})", fontsize=14)
ax1.set_ylabel("Valores Previstos Médios de r", fontsize=12)
ax1.grid(True, linestyle=":", alpha=0.7)
ax1.legend()

# ==========================================================
# GRÁFICO INFERIOR: Resíduos Padronizados
# ==========================================================

# 1. Scatter Plot dos Resíduos Padronizados
ax2.scatter(
    df_grouped["y_test"],
    df_grouped["Standardized_Residual"],
    alpha=0.7,
    s=30,
    label="|Resíduo| / DP",
)

# 2. Linha de Referência Crítica (Z-Score = 1.0)
ax2.axhline(
    y=1.0, color="red", linestyle="-", linewidth=2, label="Limite de 1 Desvio Padrão"
)
ax2.axhline(y=2.0, color="orange", linestyle="--", linewidth=1, label="Limite de 2 DP")


ax2.set_xlabel("Valores Reais de r(Y_true)", fontsize=12)
ax2.set_ylabel("Resíduo Padronizado (|Y - Ŷ| / DP)", fontsize=12)
ax2.grid(True, linestyle=":", alpha=0.7)
ax2.legend()

plt.show()

# 2 - Modelo com seleção de features não linear

## 2.1 - Selecionando features

In [ ]:
# Checa se o arquivo foi salvo corretamente
with open(caminho_features_selecionadas_non_linear_features, "rb") as f:
    loaded_data = pickle.load(f)


selected_feature_names = loaded_data["list"]

In [ ]:
selected_feature_names

In [ ]:
X_test_non_linear_selection = X_test[selected_feature_names]

In [ ]:
X_test_scaled = X_scaler.fit_transform(X_test_non_linear_selection)

In [ ]:
X_test_scaled

In [ ]:
X_test_scaled.max()

In [ ]:
X_test_scaled.min()

## 3.2 - Salvando valores normalizados

As features normalizadas serão necessárias no notebook de explicabilidade dos modelos, por isso salva-se abaixo:

In [ ]:
np.save(caminho_X_test_shap_non_linear_features, X_test_scaled)

# 3.3 - Inferência 

In [ ]:
# Carregamento do modelo (usando o caminho verificado)
best_nn_model = joblib.load(caminho_modelo_non_linear_features)

In [ ]:
# A previsão é feita na escala Z-Score (escalada)
y_pred_test_scaled = best_nn_model.predict(X_test_scaled).reshape(-1, 1)

# Faz a transformação inversa para obter os valores em 'r':
y_pred_test_original = y_scaler.inverse_transform(y_pred_test_scaled)


print("Previsão no Conjunto de Teste concluída com sucesso.")

In [ ]:
X_test_scaled.max()

In [ ]:
y_pred_test_scaled.max()

In [ ]:
X_test_scaled.min()

In [ ]:
y_pred_test_scaled.min()

In [ ]:
y_pred_test_original.max()

In [ ]:
y_pred_test_original.min()

## 3. 4 - Métricas e visualizações

In [ ]:
# Cálculo das 3 Métricas de Avaliação Finais:
mse_test = mean_squared_error(y_test_2d, y_pred_test_original)
rmse_test = root_mean_squared_error(y_test_2d, y_pred_test_original)
mae_test = mean_absolute_error(y_test_2d, y_pred_test_original)
r2_test = r2_score(y_test_2d, y_pred_test_original)

print("\n" + "=" * 50)
print("AVALIAÇÃO DE DESEMPENHO NO CONJUNTO DE TESTE")
print("=" * 50)
print(f"MSE (Erro Quadrático Médio): {mse_test:.8f}")
print(f"RMSE (Erro Absoluto Médio):   {rmse_test:.8f} (Erro médio na unidade de 'r')")
print(f"MAE (Erro Absoluto Médio):   {mae_test:.8f} (Erro médio na unidade de 'r')")
print(f"R2 (Ajuste):                 {r2_test:.4f}")
print("=" * 50)

In [ ]:
# Crie a linha de identidade X=Y
min_val = min(y_test_2d.min(), y_pred_test_original.min())
max_val = max(y_test_2d.max(), y_pred_test_original.max())
ideal_line = np.linspace(min_val, max_val, 100)

plt.figure(figsize=(8, 8))

# 1. Scatter Plot dos Resultados
plt.scatter(y_test_2d, y_pred_test_original, alpha=0.6, s=20, label="Previsões")

# 2. Linha de Identidade (Ajuste Perfeito)
plt.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

plt.title(f"Previsões vs. Valores Reais de r (R² Final: {r2_test:.4f})", fontsize=14)
plt.xlabel("Valores Reais de r", fontsize=12)
plt.ylabel("Valores Previstos de r", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.gca().set_aspect(
    "equal", adjustable="box"
)  # Garante que os eixos tenham a mesma escala
plt.show()

In [ ]:
# Calcule os resíduos (True - Predicted)
residuals = y_test_2d - y_pred_test_original

# Achata o array para 1D (o formato (N,))
residuals_1d = residuals.ravel()

plt.figure(figsize=(8, 6))

# Histograma dos resíduos
plt.hist(residuals_1d, bins=30, edgecolor="black", alpha=0.7)

# Linha vertical em zero (onde o centro do histograma deveria estar)
plt.axvline(x=0, color="red", linestyle="--", linewidth=2, label="Erro Zero")

plt.title("Distribuição dos Resíduos (Erros)", fontsize=14)
plt.xlabel("Resíduo (Real r - Previsto r)", fontsize=12)
plt.ylabel("Frequência", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.show()

## 3.4.1 - Visualização com barra de erros

Os dados utilizados associam 10 valores diferentes das features a um mesmo valor do target. O objetivo é simular os ruídos presentes em observações astronômicas.

A visualização com barras de erros também é feita abaixo utilizando-se os dados de teste.

In [ ]:
# Criando um DataFrame para facilitar a agregação
# Garantindo que y_train e y_pred_original são 1D para o DataFrame
y_test_flat = y_test_2d.ravel()
y_pred_test_flat = y_pred_test_original.ravel()

df_results = pd.DataFrame({"y_test": y_test_flat, "y_test_pred": y_pred_test_flat})

# AGREGAR: Agrupar por 'y_true' e calcular Média e Desvio Padrão para 'y_pred'
# 'y_true' (a variável agrupada) será o índice do novo DataFrame
df_grouped = (
    df_results.groupby("y_test")["y_test_pred"].agg(["mean", "std"]).reset_index()
)

# Renomear as colunas para clareza
df_grouped.columns = ["y_test", "y_test_pred_mean", "y_test_pred_std"]

# Cálculo do Resíduo Padronizado (em módulos, conforme solicitado)
# Evite a divisão por zero, substituindo desvios padrão zero por um valor pequeno (ex: 1e-6)
df_grouped["y_test_pred_std_safe"] = df_grouped["y_test_pred_std"].replace(0, 1e-6)
df_grouped["Standardized_Residual"] = (
    np.abs(df_grouped["y_test"] - df_grouped["y_test_pred_mean"])
    / df_grouped["y_test_pred_std_safe"]
)

# Exibir o resultado da agregação (opcional)
print("Dados Agrupados (Exemplo):")
print(df_grouped.head())

In [ ]:
# --- VISUALIZAÇÃO COM GRÁFICO DE DUPLO EIXO ---
# Cria um espaço de figuras com 2 linhas e 1 coluna, compartilhando o eixo X
fig, (ax1, ax2) = plt.subplots(
    2,
    1,
    figsize=(8, 10),
    sharex=True,  # Os dois gráficos compartilham o mesmo eixo X
    gridspec_kw={"hspace": 0.05},  # Reduz o espaço entre os subplots
    constrained_layout=True,
)

# ==========================================================
# GRÁFICO SUPERIOR: Previsão Média vs. Real (Com Barras de Erro)
# ==========================================================

# 1. Plotar os pontos agregados com Barras de Erro
ax1.errorbar(
    x=df_grouped["y_test"],
    y=df_grouped["y_test_pred_mean"],
    yerr=df_grouped["y_test_pred_std"],
    fmt="o",
    capsize=4,
    alpha=0.7,
    label="Previsão Média ± 1 DP",
)

# 2. Linha de Identidade (Ajuste Perfeito Y=X)
ax1.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

ax1.set_title(f"Análise de Previsão Agregada (R² Final: {r2_test:.4f})", fontsize=14)
ax1.set_ylabel("Valores Previstos Médios de r", fontsize=12)
ax1.grid(True, linestyle=":", alpha=0.7)
ax1.legend()

# ==========================================================
# GRÁFICO INFERIOR: Resíduos Padronizados
# ==========================================================

# 1. Scatter Plot dos Resíduos Padronizados
ax2.scatter(
    df_grouped["y_test"],
    df_grouped["Standardized_Residual"],
    alpha=0.7,
    s=30,
    label="|Resíduo| / DP",
)

# 2. Linha de Referência Crítica (Z-Score = 1.0)
ax2.axhline(
    y=1.0, color="red", linestyle="-", linewidth=2, label="Limite de 1 Desvio Padrão"
)
ax2.axhline(y=2.0, color="orange", linestyle="--", linewidth=1, label="Limite de 2 DP")


ax2.set_xlabel("Valores Reais de r(Y_true)", fontsize=12)
ax2.set_ylabel("Resíduo Padronizado (|Y - Ŷ| / DP)", fontsize=12)
ax2.grid(True, linestyle=":", alpha=0.7)
ax2.legend()

plt.show()

# 3 - Modelo com seleção linear de features

## 3.1 - Selecionando features

In [ ]:
# Checa se o arquivo foi salvo corretamente
with open(caminho_features_selecionadas_linear_features, "rb") as f:
    loaded_data = pickle.load(f)


selected_feature_names = loaded_data["list"]

In [ ]:
selected_feature_names

In [ ]:
X_test_linear_selection = X_test[selected_feature_names]

In [ ]:
X_test_scaled = X_scaler.fit_transform(X_test_linear_selection)

In [ ]:
X_test_scaled

In [ ]:
X_test_scaled.max()

In [ ]:
X_test_scaled.min()

## 3.2 - Salvando valores normalizados

As features normalizadas serão necessárias no notebook de explicabilidade dos modelos, por isso salva-se abaixo:

In [ ]:
np.save(caminho_X_test_shap_linear_features, X_test_scaled)

# 3.3 - Inferência 

In [ ]:
# Carregamento do modelo (usando o caminho verificado)
best_nn_model = joblib.load(caminho_modelo_linear_features)

In [ ]:
# A previsão é feita na escala Z-Score (escalada)
y_pred_test_scaled = best_nn_model.predict(X_test_scaled).reshape(-1, 1)

# Faz a transformação inversa para obter os valores em 'r':
y_pred_test_original = y_scaler.inverse_transform(y_pred_test_scaled)


print("Previsão no Conjunto de Teste concluída com sucesso.")

In [ ]:
X_test_scaled.max()

In [ ]:
y_pred_test_scaled.max()

In [ ]:
X_test_scaled.min()

In [ ]:
y_pred_test_scaled.min()

In [ ]:
y_pred_test_original.max()

In [ ]:
y_pred_test_original.min()

## 3. 4 - Métricas e visualizações

In [ ]:
# Cálculo das 3 Métricas de Avaliação Finais:
mse_test = mean_squared_error(y_test_2d, y_pred_test_original)
rmse_test = root_mean_squared_error(y_test_2d, y_pred_test_original)
mae_test = mean_absolute_error(y_test_2d, y_pred_test_original)
r2_test = r2_score(y_test_2d, y_pred_test_original)

print("\n" + "=" * 50)
print("AVALIAÇÃO DE DESEMPENHO NO CONJUNTO DE TESTE")
print("=" * 50)
print(f"MSE (Erro Quadrático Médio): {mse_test:.8f}")
print(f"RMSE (Erro Absoluto Médio):   {rmse_test:.8f} (Erro médio na unidade de 'r')")
print(f"MAE (Erro Absoluto Médio):   {mae_test:.8f} (Erro médio na unidade de 'r')")
print(f"R2 (Ajuste):                 {r2_test:.4f}")
print("=" * 50)

In [ ]:
# Crie a linha de identidade X=Y
min_val = min(y_test_2d.min(), y_pred_test_original.min())
max_val = max(y_test_2d.max(), y_pred_test_original.max())
ideal_line = np.linspace(min_val, max_val, 100)

plt.figure(figsize=(8, 8))

# 1. Scatter Plot dos Resultados
plt.scatter(y_test_2d, y_pred_test_original, alpha=0.6, s=20, label="Previsões")

# 2. Linha de Identidade (Ajuste Perfeito)
plt.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

plt.title(f"Previsões vs. Valores Reais de r (R² Final: {r2_test:.4f})", fontsize=14)
plt.xlabel("Valores Reais de r", fontsize=12)
plt.ylabel("Valores Previstos de r", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.gca().set_aspect(
    "equal", adjustable="box"
)  # Garante que os eixos tenham a mesma escala
plt.show()

In [ ]:
# Calcule os resíduos (True - Predicted)
residuals = y_test_2d - y_pred_test_original

# Achata o array para 1D (o formato (N,))
residuals_1d = residuals.ravel()

plt.figure(figsize=(8, 6))

# Histograma dos resíduos
plt.hist(residuals_1d, bins=30, edgecolor="black", alpha=0.7)

# Linha vertical em zero (onde o centro do histograma deveria estar)
plt.axvline(x=0, color="red", linestyle="--", linewidth=2, label="Erro Zero")

plt.title("Distribuição dos Resíduos (Erros)", fontsize=14)
plt.xlabel("Resíduo (Real r - Previsto r)", fontsize=12)
plt.ylabel("Frequência", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.show()

## 3.4.1 - Visualização com barra de erros

Os dados utilizados associam 10 valores diferentes das features a um mesmo valor do target. O objetivo é simular os ruídos presentes em observações astronômicas.

A visualização com barras de erros também é feita abaixo utilizando-se os dados de teste.

In [ ]:
# Criando um DataFrame para facilitar a agregação
# Garantindo que y_train e y_pred_original são 1D para o DataFrame
y_test_flat = y_test_2d.ravel()
y_pred_test_flat = y_pred_test_original.ravel()

df_results = pd.DataFrame({"y_test": y_test_flat, "y_test_pred": y_pred_test_flat})

# AGREGAR: Agrupar por 'y_true' e calcular Média e Desvio Padrão para 'y_pred'
# 'y_true' (a variável agrupada) será o índice do novo DataFrame
df_grouped = (
    df_results.groupby("y_test")["y_test_pred"].agg(["mean", "std"]).reset_index()
)

# Renomear as colunas para clareza
df_grouped.columns = ["y_test", "y_test_pred_mean", "y_test_pred_std"]

# Cálculo do Resíduo Padronizado (em módulos, conforme solicitado)
# Evite a divisão por zero, substituindo desvios padrão zero por um valor pequeno (ex: 1e-6)
df_grouped["y_test_pred_std_safe"] = df_grouped["y_test_pred_std"].replace(0, 1e-6)
df_grouped["Standardized_Residual"] = (
    np.abs(df_grouped["y_test"] - df_grouped["y_test_pred_mean"])
    / df_grouped["y_test_pred_std_safe"]
)

# Exibir o resultado da agregação (opcional)
print("Dados Agrupados (Exemplo):")
print(df_grouped.head())

In [ ]:
# --- VISUALIZAÇÃO COM GRÁFICO DE DUPLO EIXO ---
# Cria um espaço de figuras com 2 linhas e 1 coluna, compartilhando o eixo X
fig, (ax1, ax2) = plt.subplots(
    2,
    1,
    figsize=(8, 10),
    sharex=True,  # Os dois gráficos compartilham o mesmo eixo X
    gridspec_kw={"hspace": 0.05},  # Reduz o espaço entre os subplots
    constrained_layout=True,
)

# ==========================================================
# GRÁFICO SUPERIOR: Previsão Média vs. Real (Com Barras de Erro)
# ==========================================================

# 1. Plotar os pontos agregados com Barras de Erro
ax1.errorbar(
    x=df_grouped["y_test"],
    y=df_grouped["y_test_pred_mean"],
    yerr=df_grouped["y_test_pred_std"],
    fmt="o",
    capsize=4,
    alpha=0.7,
    label="Previsão Média ± 1 DP",
)

# 2. Linha de Identidade (Ajuste Perfeito Y=X)
ax1.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

ax1.set_title(f"Análise de Previsão Agregada (R² Final: {r2_test:.4f})", fontsize=14)
ax1.set_ylabel("Valores Previstos Médios de r", fontsize=12)
ax1.grid(True, linestyle=":", alpha=0.7)
ax1.legend()

# ==========================================================
# GRÁFICO INFERIOR: Resíduos Padronizados
# ==========================================================

# 1. Scatter Plot dos Resíduos Padronizados
ax2.scatter(
    df_grouped["y_test"],
    df_grouped["Standardized_Residual"],
    alpha=0.7,
    s=30,
    label="|Resíduo| / DP",
)

# 2. Linha de Referência Crítica (Z-Score = 1.0)
ax2.axhline(
    y=1.0, color="red", linestyle="-", linewidth=2, label="Limite de 1 Desvio Padrão"
)
ax2.axhline(y=2.0, color="orange", linestyle="--", linewidth=1, label="Limite de 2 DP")


ax2.set_xlabel("Valores Reais de r(Y_true)", fontsize=12)
ax2.set_ylabel("Resíduo Padronizado (|Y - Ŷ| / DP)", fontsize=12)
ax2.grid(True, linestyle=":", alpha=0.7)
ax2.legend()

plt.show()

# 4 - Sugestões para trabalhos futuros

- Criar novas features com os dados de entrada.

- Aumentar a rede de parâmetros da validação cruzada.